In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qwen2/transformers/7b-instruct/1/model.safetensors.index.json
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00003-of-00004.safetensors
/kaggle/input/qwen2/transformers/7b-instruct/1/config.json
/kaggle/input/qwen2/transformers/7b-instruct/1/merges.txt
/kaggle/input/qwen2/transformers/7b-instruct/1/LICENSE
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00001-of-00004.safetensors
/kaggle/input/qwen2/transformers/7b-instruct/1/README.md
/kaggle/input/qwen2/transformers/7b-instruct/1/tokenizer.json
/kaggle/input/qwen2/transformers/7b-instruct/1/vocab.json
/kaggle/input/qwen2/transformers/7b-instruct/1/tokenizer_config.json
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00004-of-00004.safetensors
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00002-of-00004.safetensors
/kaggle/input/qwen2/transformers/7b-instruct/1/generation_config.json


In [4]:
#!pip install --quiet transformers sentence-transformers tqdm accelerate

In [5]:
#!pip install --quiet chromadb gradio

In [6]:
#!pip install protobuf==3.20.3 --quiet --force-reinstall

In [ ]:
#!pip install textstat

In [9]:
#!pip install --upgrade datasets

In [3]:
import torch
print("GPU0 free:", torch.cuda.mem_get_info(0)[0] / 1024**3, "GB")
print("GPU1 free:", torch.cuda.mem_get_info(1)[0] / 1024**3, "GB")


GPU0 free: 14.6407470703125 GB
GPU1 free: 14.6407470703125 GB


In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


In [8]:
import os, json, torch, time
from datasets import load_dataset
import chromadb
from chromadb.utils import embedding_functions
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
from typing import List, Tuple, Dict

# Paths
CHROMA_PATH = "/kaggle/working/chroma_db"
QWEN_PATH = "/kaggle/input/qwen2/transformers/7b-instruct/1"  # Change if different
COLLECTION_NAME = "health_rag_collection"

# Models
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Agent settings
TOPK_INITIAL = 5
TOPK_EXPANDED = 10
CONF_THRESHOLD = 0.75


In [10]:
# Core datasets
ds_health = load_dataset("katielink/healthsearchqa", "all_data")       # Layman queries
ds_mediqa = load_dataset("medarc/consumer_health_questions")               # Patient-oriented answers
ds_flash = load_dataset("medalpaca/medical_meadow_medical_flashcards")
print(ds_flash['train'][0])

print("Datasets Loaded:")



README.md: 0.00B [00:00, ?B/s]

all.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/4436 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/726 [00:00<?, ?B/s]

data/train-00000-of-00001-75c4fb7f8b48a5(…):   0%|          | 0.00/263k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/150 [00:00<?, ? examples/s]

README.md: 0.00B [00:00, ?B/s]

medical_meadow_wikidoc_medical_flashcard(…):   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/33955 [00:00<?, ? examples/s]

{'input': 'What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?', 'output': 'Very low Mg2+ levels correspond to low PTH levels which in turn results in low Ca2+ levels.', 'instruction': 'Answer this question truthfully'}
Datasets Loaded:


In [11]:
client = chromadb.PersistentClient(path=CHROMA_PATH)
embed_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)
collection = client.get_or_create_collection(name=COLLECTION_NAME, embedding_function=embed_fn)


E0000 00:00:1763892134.891983     135 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763892134.935505     135 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
print("HealthSearch example:", ds_health['train'][0])
print("MediQA/consumer example:", ds_mediqa['train'][0])
print("Flashcard example:", ds_flash['train'][0])


HealthSearch example: {'id': 1.0, 'question': 'Are benign brain tumors serious?'}
MediQA/consumer example: {'id': 0, 'inputs': 'SUBJECT: who and where to get cetirizine - D MESSAGE: I need/want to know who manufscturs Cetirizine. My Walmart is looking for a new supply and are not getting the recent', 'target': 'Who manufactures cetirizine?'}
Flashcard example: {'input': 'What is the relationship between very low Mg2+ levels, PTH levels, and Ca2+ levels?', 'output': 'Very low Mg2+ levels correspond to low PTH levels which in turn results in low Ca2+ levels.', 'instruction': 'Answer this question truthfully'}


In [13]:
print("Chroma collection size:", collection.count())

Chroma collection size: 0


In [14]:
if collection.count() == 0:
    docs, metas, ids = [], [], []
    idx = 0  # unique id tracking

    # 🔹 1. Ingest medical Q&A from flashcard dataset
    for row in ds_flash['train']:
        text = row.get('output', '')
        if text and len(text.strip()) > 30:
            docs.append(text.strip())
            metas.append({"source": "flashcard_dataset"})
            ids.append(str(idx))
            idx += 1
        if idx >= 2000: break  # Limit ingestion

    # 🔹 2. Ingest refined patient question dataset
    for row in ds_mediqa['train']:
        text = row.get('target', '')  # use cleaned version
        if text and len(text.strip()) > 30:
            docs.append(text.strip())
            metas.append({"source": "consumer_health"})
            ids.append(str(idx))
            idx += 1
        if idx >= 3500: break  # Combined target cap

    # 🚀 Ingest into ChromaDB
    collection.add(documents=docs, metadatas=metas, ids=ids)
    print(f"🔥 Successfully ingested {len(ids)} documents.")

else:
    print(f"🟢 ChromaDB already contains {collection.count()} documents.")


🔥 Successfully ingested 2971 documents.


In [15]:
print("📦 Final ChromaDB document count:", collection.count())


📦 Final ChromaDB document count: 2971


In [16]:
print("🔍 Retrieval test:")
results = collection.query(query_texts=["diabetes surgery complication"], n_results=3)
for doc in results['documents'][0]:
    print("- ", doc[:150], "...")


🔍 Retrieval test:
-  What are the treatments for diabetes and can it be reversed? ...
-  What are the complications of type 1 diabetes? ...
-  What is the latest research on pancreas transplants for type one diabetes? ...


In [17]:
tokenizer = AutoTokenizer.from_pretrained(QWEN_PATH)
model = AutoModelForCausalLM.from_pretrained(QWEN_PATH, torch_dtype="auto", device_map="auto")
model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb):

In [25]:
def chat(system, user, mode="answer", max_tokens=512):
    # 🎯 Choose decoding strategy
    do_sample = True if mode == "evaluation" else False
    temperature = 0.3 if mode == "evaluation" else None

    # 🚫 No ellipsis here
    messages = [
        {"role": "system", "content": str(system)},
        {"role": "user", "content": str(user)}
    ]

    # 👇 CORRECT: remove ellipsis and use proper message input
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=do_sample,
            temperature=temperature,
            top_p=None  # Disable nucleus sampling if not using sampling mode
        )

    return tokenizer.decode(
        output[0, inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    ).strip()


In [19]:
print(chat("You are assistant.", "Hello!"))


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Hello! How can I assist you today?


In [26]:
from time import time
import pandas as pd
import json

# ===== Final Config Parameters =====
TOPK_INITIAL = 5
TOPK_EXPANDED = 15
CONF_THRESHOLD = 0.4   # Force retry if < 0.4
FAST_MODE_TOKENS = 420  # More tokens for deep reasoning

# ==================================
# 1️⃣ SAFE RETRIEVAL + CONTEXT
# ==================================
def retrieve(query, k=TOPK_INITIAL):
    result = collection.query(query_texts=[query], n_results=k, include=["distances"])

    docs_raw = result.get("documents", [[]])
    metas_raw = result.get("metadatas", [[]])
    dists_raw = result.get("distances", [[]])

    if not docs_raw or docs_raw[0] is None:
        return [], [0.0]

    docs = list(zip(docs_raw[0], metas_raw[0]))
    sims = [(1 - d) if d is not None else 0.5 for d in dists_raw[0]]
    return docs, sims


def format_context(docs):
    if not docs:
        return "No relevant documents found."
    return "\n\n".join([f"[Doc {i+1}] {str(d)[:400]}" for i, (d, _) in enumerate(docs)])


def safe_preview_text(ans):
    return ((str(ans) if ans is not None else "No response generated")[:150] + "...")

# ==================================
# 2️⃣ AGENT HELPERS (No temp used here)
# ==================================
def generate_clarify(query):
    return chat("If query is unclear, ask ONE follow-up question.", f"Query: {query}\nClarify:")

def reformulate_query(query):
    return chat("Convert patient query to medical terminology.", f"Original query: {query}\nReformulate:")

def risk_awareness(response):
    return chat("Provide general risk awareness (not advice).", f"Response: {response}")

def doctor_questions(response):
    return chat("Suggest 1-2 general questions for doctor.", f"Response: {response}")

# ==================================
# 3️⃣ MAIN AGENTIC RAG FUNCTION
# ==================================
def agentic_rag(query, mode="patient"):
    max_tokens = FAST_MODE_TOKENS
    retry_enabled = True  # Retry allowed if low confidence

    # 1. Clarify
    clarify = generate_clarify(query)
    if "?" in clarify:
        query = clarify

    # 2. Reformulate
    med_query = reformulate_query(query)

    # 3. Retrieve docs
    docs, sims = retrieve(med_query)
    retrieval_sim = max(sims) if sims else 0.5

    # 4. Generate response
    sys_prompt = "Answer using only given medical context. Be accurate and clear."
    user_prompt = f"Q: {query}\nContext:\n{format_context(docs)}"
    answer = chat(sys_prompt, user_prompt, max_tokens=max_tokens)  # ❌ No temperature

    # 5. Self-evaluate
    eval_sys = "Rate JSON {'score':0-1,'needs_retry':bool}"
    eval_res = chat(eval_sys, f"Q: {query}\nAnswer: {answer}")
    try:
        ev = json.loads(eval_res)
    except:
        ev = {"score": 0.5, "needs_retry": True}

    # 6. Final confidence using new formula
    llm_conf = ev.get("score", 0.5)
    final_conf = round(0.5 * llm_conf + 0.5 * retrieval_sim, 2)

    # 7. Retry if needed
    if retry_enabled and final_conf < CONF_THRESHOLD:
        docs, sims = retrieve(med_query, TOPK_EXPANDED)
        answer = chat(sys_prompt, f"Q: {query}\nContext:\n{format_context(docs)}", max_tokens=max_tokens)

    # 8. Final patient-style response
    if mode == "patient":
        answer = chat(
            "Rewrite in supportive tone. Avoid medical advice. Encourage doctor discussion.",
            answer,
            max_tokens=max_tokens
        )
        answer += "\n\n⚠️ " + risk_awareness(answer)
        answer += "\n\n🩺 Suggested questions:\n- " + doctor_questions(answer)

    ev["final_confidence"] = final_conf
    return answer, ev

# ==================================
# 4️⃣ EVALUATION CODE
# ==================================
def evaluate_system(queries):
    results = []
    for q in queries:
        start = time()
        ans, ev = agentic_rag(q, mode="patient")
        elapsed = time() - start

        results.append({
            "query": q,
            "confidence_score": ev.get("final_confidence", 0),
            "needed_retry": ev.get("needs_retry", False),
            "response_time_sec": round(elapsed, 2),
            "response_preview": safe_preview_text(ans)
        })
    return results

# ==================================
# 5️⃣ TEST IT
# ==================================
test_queries = [
    "I have high blood pressure and I’m going for surgery. Should I worry?",
    "I have diabetes and I'm scheduled for cataract surgery. Is it risky?",
    "I have asthma but I often ignore inhalers. Can it affect anesthesia?",
    "I take multiple heart medications. Should I inform my doctor before treatment?",
    "I often skip sugar check-ups. Will it slow recovery after surgery?"
]

fast_results = evaluate_system(test_queries)
df_fast = pd.DataFrame(fast_results)

# Fix missing values
df_fast["confidence_score"] = df_fast["confidence_score"].fillna(0)
df_fast["needed_retry"] = df_fast["needed_retry"].fillna(False)

summary_fast = {
    "avg_confidence": round(df_fast["confidence_score"].mean(), 3),
    "retry_rate_%": round((df_fast["needed_retry"].sum() / len(df_fast)) * 100, 2),
    "avg_response_time_sec": round(df_fast["response_time_sec"].mean(), 2)
}

# Final output
df_fast, summary_fast

The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=

(                                               query  confidence_score  \
 0  I have high blood pressure and I’m going for s...              0.50   
 1  I have diabetes and I'm scheduled for cataract...              0.50   
 2  I have asthma but I often ignore inhalers. Can...              0.25   
 3  I take multiple heart medications. Should I in...              0.50   
 4  I often skip sugar check-ups. Will it slow rec...              0.25   
 
    needed_retry  response_time_sec  \
 0         False              47.32   
 1         False              64.06   
 2          True              67.74   
 3         False              40.51   
 4          True             114.45   
 
                                     response_preview  
 0  Managing high blood pressure before surgery is...  
 1  It's absolutely crucial to consider the unique...  
 2  It's reassuring to know that using an asthma i...  
 3  Absolutely, it's incredibly important to share...  
 4  Regular monitoring of your b

In [28]:
def interactive_chat():
    print("🩺 Agentic RAG Medical Assistant (Notebook Mode)")
    print("Type 'exit' to stop.\n")
    
    while True:
        query = input("🔎 Enter your query: ")  # User types directly in console
        
        if query.lower() in ["exit", "quit", "stop"]:
            print("\n👋 Exiting... Stay healthy!")
            break
        
        start = time()
        answer, ev = agentic_rag(query, mode="patient")
        elapsed = round(time() - start, 2)

        confidence = ev.get("final_confidence", None)
        retry = ev.get("needs_retry", False)

        print("\n💬 Response:")
        print(answer)
        print(f"\n📈 Confidence: {confidence}")
        print(f"🔁 Retry Triggered: {retry}")
        print(f"⏱ Response Time: {elapsed} sec\n")
        print("-" * 80)

# Run the console chat
interactive_chat()


🩺 Agentic RAG Medical Assistant (Notebook Mode)
Type 'exit' to stop.



🔎 Enter your query:  i have a burning sensation in my eyes and high temperature around 40 degree celsius and after few hours i completely lost my sense of smell... what to do now


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=


💬 Response:
It sounds like you might be concerned about some potential symptoms that aren't currently documented. It's really important to pay attention to your health and any changes you notice. If you're experiencing symptoms such as a cough, difficulty breathing, or body aches, I encourage you to discuss these with a healthcare professional. They can provide personalized guidance and help ensure you receive the care you need. Don't hesitate to reach out to your doctor for a thorough assessment. Your health is key, and taking proactive steps by consulting with a medical expert can offer peace of mind and appropriate support.

⚠️ Absolutely, staying aware of your overall health and being proactive about any changes or concerns is crucial. Regular check-ups with healthcare professionals are recommended to monitor your well-being and address any issues early on. This includes keeping track of your physical, mental, and emotional health. If you experience any unusual symptoms, it's wise

🔎 Enter your query:  i have a slight headache and running nose what to do 


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



💬 Response:
In situations where you're unable to locate pertinent documents, it's highly recommended to consult with a healthcare provider if you're experiencing symptoms that need immediate attention or clarification. For minor concerns that can typically be managed at home, applying basic self-care strategies might suffice. However, for more complex or concerning health issues, seeking professional medical guidance is crucial. This ensures that any necessary treatments or further diagnostics are provided by qualified medical professionals, helping to address your health needs effectively and safely. Always feel free to discuss your symptoms and concerns openly with your doctor to receive personalized advice tailored to your specific situation.

⚠️ Risk awareness in health situations involves being proactive about your well-being and understanding the potential consequences of various actions. Here are some key points to consider:

1. **Symptom Recognition**: Knowing when a symptom r

🔎 Enter your query:  exit



👋 Exiting... Stay healthy!


In [ ]:
import gradio as gr

def gradio_interface(user_query):
    start = time()
    
    # Run Fast Agentic RAG
    answer, ev = agentic_rag(user_query, mode="patient")
    elapsed = round(time() - start, 2)

    confidence = ev.get("final_confidence", None)
    retry = ev.get("needs_retry", False)

    return answer, confidence, retry, elapsed


# ---------------------------------
# 🖥️ Gradio UI Setup
# ---------------------------------
with gr.Blocks() as demo:
    gr.Markdown("""
    # 🩺 Agentic RAG Health Support System  
    *Non-advisory medical information assistant (Portfolio Demo)*  
    **Features:**
    - Converts patient queries into medical context
    - Retrieves relevant context
    - Self-evaluates responses
    - Reformulates in patient-friendly mode
    - Uses retry if confidence is low
    """)

    with gr.Row():
        user_query = gr.Textbox(
            label="🔎 Enter your question",
            placeholder="e.g., I have diabetes and I'm having eye surgery soon...",
            lines=3
        )

    submit_button = gr.Button("🚀 Get Response")

    with gr.Row():
        answer_output = gr.Textbox(label="💬 AI Response", lines=7)
    
    with gr.Row():
        confidence_output = gr.Number(label="📈 Confidence Score")
        retry_output = gr.Checkbox(label="🔁 Retry Triggered?")
        time_output = gr.Number(label="⏱ Response Time (sec)")

    submit_button.click(
        fn=gradio_interface,
        inputs=user_query,
        outputs=[answer_output, confidence_output, retry_output, time_output]
    )

demo.launch()
